## Project 3
Batch normalization, dropout에 따른 모델의 성능 변화 이해

학번: 20191300
이름: 남유정

In [1]:
import os, random
import keras
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.utils import np_utils
import numpy as np
import tensorflow as tf
os.path.expanduser = lambda path: './'

In [2]:
batch_size = 128
num_classes = 10
epochs = 60

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


## Define Model - activation function = 'relu'
without batch_normalization, 
dropout = 0.2

In [3]:
from keras import backend as K

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dense(512, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 10)                5130      
                                                                 
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


## Start Training

In [4]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)

Epoch 1/60
375/375 [==============================] - 5s 11ms/step - loss: 1.0757 - accuracy: 0.6890 - val_loss: 0.7244 - val_accuracy: 0.7667
Epoch 2/60
375/375 [==============================] - 3s 9ms/step - loss: 0.6513 - accuracy: 0.7920 - val_loss: 0.5930 - val_accuracy: 0.8067
Epoch 3/60
375/375 [==============================] - 3s 8ms/step - loss: 0.5648 - accuracy: 0.8127 - val_loss: 0.5371 - val_accuracy: 0.8224
Epoch 4/60
375/375 [==============================] - 4s 10ms/step - loss: 0.5229 - accuracy: 0.8243 - val_loss: 0.5100 - val_accuracy: 0.8243
Epoch 5/60
375/375 [==============================] - 3s 8ms/step - loss: 0.4963 - accuracy: 0.8315 - val_loss: 0.4851 - val_accuracy: 0.8328
Epoch 6/60
375/375 [==============================] - 3s 9ms/step - loss: 0.4755 - accuracy: 0.8368 - val_loss: 0.4837 - val_accuracy: 0.8326
Epoch 7/60
375/375 [==============================] - 3s 8ms/step - loss: 0.4606 - accuracy: 0.8420 - val_loss: 0.4580 - val_accuracy: 0.8424
Epoc

375/375 [==============================] - 5s 13ms/step - loss: 0.2801 - accuracy: 0.9013 - val_loss: 0.3392 - val_accuracy: 0.8798
Epoch 59/60
375/375 [==============================] - 5s 14ms/step - loss: 0.2793 - accuracy: 0.9024 - val_loss: 0.3467 - val_accuracy: 0.8788
Epoch 60/60
375/375 [==============================] - 6s 16ms/step - loss: 0.2765 - accuracy: 0.9020 - val_loss: 0.3321 - val_accuracy: 0.8813


## Calculate accuracy

In [5]:
metrics = model.evaluate(x_test, y_test) #returns loss and accuracy
print(metrics[1])
print(f'Accuracy: {metrics[1]*100:.2f}%\n')

313/313 [==============================] - 2s 7ms/step - loss: 0.3551 - accuracy: 0.8746
0.8745999932289124
Accuracy: 87.46%



## Define Model - activation function = 'relu'
with batch_normalization, 
dropout = 0.2

In [6]:
import random, os
os.environ['PYTHONHASHSEED']='0'
random.seed(123)
np.random.seed(123)
tf.random.set_seed(123)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,    inter_op_parallelism_threads=1,
                                                               allow_soft_placement=True, device_count = {'CPU': 1}))
from tensorflow.python.keras import backend as K
K.set_session(sess)



kernel_initializer='glorot_uniform'
activation_function = 'relu'

with tf.device('/cpu:0'):
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dense_4 (Dense)             (None, 512)               262656    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                      

In [7]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)

Epoch 1/60
375/375 [==============================] - 8s 19ms/step - loss: 0.7246 - accuracy: 0.7478 - val_loss: 0.5106 - val_accuracy: 0.8233
Epoch 2/60
375/375 [==============================] - 6s 15ms/step - loss: 0.5081 - accuracy: 0.8192 - val_loss: 0.4165 - val_accuracy: 0.8512
Epoch 3/60
375/375 [==============================] - 5s 13ms/step - loss: 0.4561 - accuracy: 0.8368 - val_loss: 0.3869 - val_accuracy: 0.8626
Epoch 4/60
375/375 [==============================] - 5s 13ms/step - loss: 0.4231 - accuracy: 0.8493 - val_loss: 0.3697 - val_accuracy: 0.8677
Epoch 5/60
375/375 [==============================] - 5s 13ms/step - loss: 0.4016 - accuracy: 0.8557 - val_loss: 0.3610 - val_accuracy: 0.8710
Epoch 6/60
375/375 [==============================] - 5s 12ms/step - loss: 0.3845 - accuracy: 0.8634 - val_loss: 0.3481 - val_accuracy: 0.8741
Epoch 7/60
375/375 [==============================] - 5s 12ms/step - loss: 0.3694 - accuracy: 0.8678 - val_loss: 0.3465 - val_accuracy: 0.8753

Epoch 58/60
375/375 [==============================] - 9s 24ms/step - loss: 0.1577 - accuracy: 0.9412 - val_loss: 0.3181 - val_accuracy: 0.8952
Epoch 59/60
375/375 [==============================] - 8s 20ms/step - loss: 0.1545 - accuracy: 0.9428 - val_loss: 0.3153 - val_accuracy: 0.8943
Epoch 60/60
375/375 [==============================] - 8s 20ms/step - loss: 0.1575 - accuracy: 0.9430 - val_loss: 0.3221 - val_accuracy: 0.8937


In [8]:
metrics = model.evaluate(x_test, y_test) #returns loss and accuracy
print(metrics[1])
print(f'Accuracy: {metrics[1]*100:.2f}%\n')

313/313 [==============================] - 2s 6ms/step - loss: 0.3466 - accuracy: 0.8885
0.8884999752044678
Accuracy: 88.85%



## Define Model - activation function = 'relu'
without batch_normalization, 
dropout = 0.5

In [9]:
with tf.device('/cpu:0'):
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.5))
    #model.add(BatchNormalization())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    #model.add(BatchNormalization())
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 512)               401920    
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_7 (Dense)             (None, 512)               262656    
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_8 (Dense)             (None, 10)                5130      
                                                                 
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


In [10]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)

Epoch 1/60
375/375 [==============================] - 6s 14ms/step - loss: 1.4242 - accuracy: 0.5079 - val_loss: 0.8076 - val_accuracy: 0.7228
Epoch 2/60
375/375 [==============================] - 5s 13ms/step - loss: 0.8997 - accuracy: 0.6824 - val_loss: 0.6740 - val_accuracy: 0.7628
Epoch 3/60
375/375 [==============================] - 5s 12ms/step - loss: 0.7753 - accuracy: 0.7296 - val_loss: 0.6132 - val_accuracy: 0.7813
Epoch 4/60
375/375 [==============================] - 4s 11ms/step - loss: 0.7043 - accuracy: 0.7532 - val_loss: 0.5728 - val_accuracy: 0.7990
Epoch 5/60
375/375 [==============================] - 4s 11ms/step - loss: 0.6601 - accuracy: 0.7720 - val_loss: 0.5393 - val_accuracy: 0.8117
Epoch 6/60
375/375 [==============================] - 4s 10ms/step - loss: 0.6249 - accuracy: 0.7848 - val_loss: 0.5182 - val_accuracy: 0.8192
Epoch 7/60
375/375 [==============================] - 4s 11ms/step - loss: 0.5964 - accuracy: 0.7903 - val_loss: 0.5003 - val_accuracy: 0.8244

Epoch 58/60
375/375 [==============================] - 6s 15ms/step - loss: 0.3546 - accuracy: 0.8723 - val_loss: 0.3350 - val_accuracy: 0.8794
Epoch 59/60
375/375 [==============================] - 5s 14ms/step - loss: 0.3548 - accuracy: 0.8731 - val_loss: 0.3350 - val_accuracy: 0.8787
Epoch 60/60
375/375 [==============================] - 7s 18ms/step - loss: 0.3507 - accuracy: 0.8747 - val_loss: 0.3334 - val_accuracy: 0.8804


In [11]:
metrics = model.evaluate(x_test, y_test) #returns loss and accuracy
print(metrics[1])
print(f'Accuracy: {metrics[1]*100:.2f}%\n')

313/313 [==============================] - 2s 6ms/step - loss: 0.3617 - accuracy: 0.8710
0.8709999918937683
Accuracy: 87.10%



## Define Model - activation function = 'relu'
with batch_normalization, 
dropout = 0.5

In [12]:
with tf.device('/cpu:0'):
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 512)               401920    
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 batch_normalization_2 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dense_10 (Dense)            (None, 512)               262656    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 batch_normalization_3 (Batc  (None, 512)              2048      
 hNormalization)                                      

In [13]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)

Epoch 1/60
375/375 [==============================] - 8s 18ms/step - loss: 1.0723 - accuracy: 0.6332 - val_loss: 0.6045 - val_accuracy: 0.7862
Epoch 2/60
375/375 [==============================] - 8s 20ms/step - loss: 0.7022 - accuracy: 0.7524 - val_loss: 0.4904 - val_accuracy: 0.8238
Epoch 3/60
375/375 [==============================] - 8s 22ms/step - loss: 0.6192 - accuracy: 0.7824 - val_loss: 0.4612 - val_accuracy: 0.8330
Epoch 4/60
375/375 [==============================] - 7s 18ms/step - loss: 0.5726 - accuracy: 0.7970 - val_loss: 0.4396 - val_accuracy: 0.8424
Epoch 5/60
375/375 [==============================] - 6s 15ms/step - loss: 0.5389 - accuracy: 0.8079 - val_loss: 0.4220 - val_accuracy: 0.8478
Epoch 6/60
375/375 [==============================] - 5s 13ms/step - loss: 0.5185 - accuracy: 0.8158 - val_loss: 0.4092 - val_accuracy: 0.8522
Epoch 7/60
375/375 [==============================] - 5s 13ms/step - loss: 0.4974 - accuracy: 0.8203 - val_loss: 0.4003 - val_accuracy: 0.8565

Epoch 58/60
375/375 [==============================] - 5s 14ms/step - loss: 0.2966 - accuracy: 0.8910 - val_loss: 0.3103 - val_accuracy: 0.8877
Epoch 59/60
375/375 [==============================] - 6s 15ms/step - loss: 0.2923 - accuracy: 0.8924 - val_loss: 0.3100 - val_accuracy: 0.8882
Epoch 60/60
375/375 [==============================] - 6s 17ms/step - loss: 0.2953 - accuracy: 0.8924 - val_loss: 0.3091 - val_accuracy: 0.8867


In [14]:
metrics = model.evaluate(x_test, y_test) #returns loss and accuracy
print(metrics[1])
print(f'Accuracy: {metrics[1]*100:.2f}%\n')

313/313 [==============================] - 2s 6ms/step - loss: 0.3322 - accuracy: 0.8801
0.8801000118255615
Accuracy: 88.01%



## Define Model - activation function = 'relu'
without batch_normalization, 
dropout = 0.8

In [15]:
with tf.device('/cpu:0'):
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.8))
    #model.add(BatchNormalization())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.8))
    #model.add(BatchNormalization())
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 512)               262656    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_14 (Dense)            (None, 10)                5130      
                                                                 
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


In [16]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)

Epoch 1/60
375/375 [==============================] - 8s 19ms/step - loss: 1.9845 - accuracy: 0.2933 - val_loss: 1.2229 - val_accuracy: 0.6821
Epoch 2/60
375/375 [==============================] - 7s 18ms/step - loss: 1.3903 - accuracy: 0.4896 - val_loss: 0.8926 - val_accuracy: 0.7154
Epoch 3/60
375/375 [==============================] - 7s 19ms/step - loss: 1.1444 - accuracy: 0.5727 - val_loss: 0.7770 - val_accuracy: 0.7351
Epoch 4/60
375/375 [==============================] - 6s 17ms/step - loss: 1.0230 - accuracy: 0.6199 - val_loss: 0.7176 - val_accuracy: 0.7487
Epoch 5/60
375/375 [==============================] - 6s 15ms/step - loss: 0.9502 - accuracy: 0.6469 - val_loss: 0.6772 - val_accuracy: 0.7648
Epoch 6/60
375/375 [==============================] - 5s 13ms/step - loss: 0.8945 - accuracy: 0.6714 - val_loss: 0.6489 - val_accuracy: 0.7703
Epoch 7/60
375/375 [==============================] - 4s 12ms/step - loss: 0.8506 - accuracy: 0.6877 - val_loss: 0.6237 - val_accuracy: 0.7829

Epoch 58/60
375/375 [==============================] - 4s 12ms/step - loss: 0.4929 - accuracy: 0.8281 - val_loss: 0.3992 - val_accuracy: 0.8554
Epoch 59/60
375/375 [==============================] - 4s 11ms/step - loss: 0.4898 - accuracy: 0.8296 - val_loss: 0.3983 - val_accuracy: 0.8577
Epoch 60/60
375/375 [==============================] - 4s 10ms/step - loss: 0.4879 - accuracy: 0.8298 - val_loss: 0.3970 - val_accuracy: 0.8570


In [17]:
metrics = model.evaluate(x_test, y_test) #returns loss and accuracy
print(metrics[1])
print(f'Accuracy: {metrics[1]*100:.2f}%\n')

313/313 [==============================] - 1s 5ms/step - loss: 0.4199 - accuracy: 0.8499
0.8499000072479248
Accuracy: 84.99%



## Define Model - activation function = 'relu'
with batch_normalization, 
dropout = 0.8

In [18]:
with tf.device('/cpu:0'):
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(784,)))
    model.add(Dropout(0.8))
    model.add(BatchNormalization())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.8))
    model.add(BatchNormalization())
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 512)               401920    
                                                                 
 dropout_8 (Dropout)         (None, 512)               0         
                                                                 
 batch_normalization_4 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dense_16 (Dense)            (None, 512)               262656    
                                                                 
 dropout_9 (Dropout)         (None, 512)               0         
                                                                 
 batch_normalization_5 (Batc  (None, 512)              2048      
 hNormalization)                                      

In [19]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2)

Epoch 1/60
375/375 [==============================] - 5s 12ms/step - loss: 2.0214 - accuracy: 0.3171 - val_loss: 1.0129 - val_accuracy: 0.7117
Epoch 2/60
375/375 [==============================] - 5s 13ms/step - loss: 1.2957 - accuracy: 0.5311 - val_loss: 0.7767 - val_accuracy: 0.7398
Epoch 3/60
375/375 [==============================] - 7s 20ms/step - loss: 1.0742 - accuracy: 0.6075 - val_loss: 0.7008 - val_accuracy: 0.7458
Epoch 4/60
375/375 [==============================] - 8s 21ms/step - loss: 0.9628 - accuracy: 0.6473 - val_loss: 0.6694 - val_accuracy: 0.7607
Epoch 5/60
375/375 [==============================] - 8s 22ms/step - loss: 0.8943 - accuracy: 0.6715 - val_loss: 0.6345 - val_accuracy: 0.7753
Epoch 6/60
375/375 [==============================] - 8s 21ms/step - loss: 0.8474 - accuracy: 0.6914 - val_loss: 0.6176 - val_accuracy: 0.7793
Epoch 7/60
375/375 [==============================] - 6s 17ms/step - loss: 0.8157 - accuracy: 0.7023 - val_loss: 0.5979 - val_accuracy: 0.7864

Epoch 58/60
375/375 [==============================] - 10s 27ms/step - loss: 0.5026 - accuracy: 0.8238 - val_loss: 0.4095 - val_accuracy: 0.8563
Epoch 59/60
375/375 [==============================] - 8s 21ms/step - loss: 0.4992 - accuracy: 0.8253 - val_loss: 0.4055 - val_accuracy: 0.8579
Epoch 60/60
375/375 [==============================] - 6s 17ms/step - loss: 0.4988 - accuracy: 0.8258 - val_loss: 0.4029 - val_accuracy: 0.8577


In [20]:
metrics = model.evaluate(x_test, y_test) #returns loss and accuracy
print(metrics[1])
print(f'Accuracy: {metrics[1]*100:.2f}%\n')

313/313 [==============================] - 2s 6ms/step - loss: 0.4252 - accuracy: 0.8487
0.8486999869346619
Accuracy: 84.87%



Dropout이 같을 때, batch normalization이 있을 때, 없을 때보다 더 정확도가 올라가는 것을 볼 수 있었다. 

또한, batch normalization이 있을 때와 없을 때 둘 다 dropout이 클수록 정확도는 낮아지는 경향성을 보였다. 

Dropout은 overfitting을 방지하기 위한 방법으로, hidden layer의 일부 뉴런을 deactivated 되도록 하는 방식이다. 모든 뉴런의 결과를 사용하지 않으므로 정확도가 떨어지지만, overfitting을 방지하는 효과가 나는 것이다. 
비활성화 하는 뉴런의 수가 많아질수록 정확도가 감소하게 되므로, 위 코드의 결과의 경황성과 일치한다. 

Batch normalization을 통해, gradient가 global optima를 찾지 못하고 local minima에 빠지거나 vanish하는 현상을 해소할 수 있다. 이를 통해 정확도를 높이는 효과를 나타낼 수 있으므로, 위 코드의 경향성과 일치한다. 